In [1]:
import pandas as pd
import numpy as np
import os
#from scipy.stats import chi2

In [2]:
def calculate_midpoint(input_str):
    # 切分输入字符串，提取数字部分
    parts = input_str.split(':')[-1].split('-')
    
    # 提取数字calculate_midpoint
    start_num = int(parts[0])
    end_num = int(parts[1])
    
    # 计算中点
    midpoint = (start_num + end_num) // 2
    
    return midpoint

# 测试
input_str = "chr10:100006245-100006832"
midpoint = calculate_midpoint(input_str)
print("Midpoint:", midpoint)

vectorized_calculate_midpoint = np.vectorize(calculate_midpoint)

Midpoint: 100006538


In [3]:
caQTL_dir = '/CIMA/Result/caQTL_tensorqtl/'
caQTL_temp_dir = '/CIMA/Data/SMR_caQTL_besd/result_for_besd/'
caQTL_output_dir = '/CIMA/Data/SMR_caQTL_besd/besd/'
smr = '/media/scPBMC1_AnalysisDisk1/huangzhuoli/Script_HPC/software_gaoyue/SMR/smr-1.3.1-linux-x86_64/smr-1.3.1'

In [16]:
CT_list = pd.read_csv('/CIMA/Data/20241230_xQTL_cell_sample_number/20241230_celltype_70_scATAC.csv')

In [17]:
CT_list['final_annotation'].to_csv('/CIMA/Script/overlap_42_CT.txt',sep='\t',header=False,index=False)

In [7]:
bim = pd.read_csv('/CIMA/genetics/qc/10.maf01.bim', header=None, sep='\t')
bim.columns = ['chr','variant_id','dis','pos','A1','A2']

In [14]:
for celltype in CT_list['final_annotation'].to_list():
    print('processing_'+celltype)
    caPeak = pd.read_csv('/CIMA/Result/20250108_cis_caQTL_studywise_sig.csv')
    caPeak = caPeak[caPeak['celltype'] == celltype]
    caPeak = caPeak['phenotype_id'].unique()

    parquet_files = [f for f in os.listdir(f'{caQTL_dir}{celltype}') if f.endswith('.parquet')]

    # 初始化一个空的 DataFrame 来存储结果
    caQTL_match = pd.DataFrame()

    # 循环遍历每个.parquet文件
    for file in parquet_files:
        # 读取.parquet文件为 DataFrame
        file_path = os.path.join(f'{caQTL_dir}{celltype}', file)
        caQTL_result = pd.read_parquet(file_path)    
        # 过滤数据并合并
        caQTL_result_match = caQTL_result[caQTL_result['phenotype_id'].isin(caPeak)]
        caQTL_match = pd.concat([caQTL_match, caQTL_result_match])
    
    #make besd
    caQTL_match_besd = caQTL_match.loc[:,['variant_id','phenotype_id','slope','slope','pval_nominal','pval_nominal']]
    caQTL_match_besd.columns = ['SNP','gene','beta','t-stat','p-value','FDR']
    caQTL_match_besd ['t-stat'] = 'NA'
    caQTL_match_besd ['FDR'] = 'NA'
    caQTL_match_besd.to_csv(f'{caQTL_temp_dir}/{celltype}_caQTL_for_besd.txt', index=False, sep='\t')
    
    caqtl = f'{caQTL_temp_dir}/{celltype}_caQTL_for_besd.txt'
    gi = f'{smr} --eqtl-summary {caqtl} --matrix-eqtl-format --make-besd --out {caQTL_output_dir}{celltype}'
    os.system(gi)
    
    print('updating_'+celltype)
    esi = pd.read_csv(f'{caQTL_output_dir}{celltype}.esi', sep='\t', header=None)
    esi.columns = ['chr','variant_id','dis','pos','A1','A2','af']
    esi = esi[['variant_id']]
    esi = pd.merge(esi, bim.loc[:,['chr','variant_id','dis','pos','A1','A2']], left_on='variant_id', right_on='variant_id')

    af_df = caQTL_match.loc[:,['variant_id','af']]
    af_df_dup = af_df.duplicated(subset='variant_id', keep='first')
    af_df = af_df.loc[~af_df_dup]

    esi = pd.merge(esi, af_df, left_on='variant_id', right_on='variant_id')
    esi = esi.loc[:,['chr','variant_id','dis','pos','A1','A2','af']]
    esi.to_csv(f'{caQTL_output_dir}{celltype}_update.esi', sep='\t', header=None, index=False)

    #linux command
    gi = f'{smr} --beqtl-summary {caQTL_output_dir}{celltype} --update-esi {caQTL_output_dir}{celltype}_update.esi'
    os.system(gi)

    #updata_epi
    epi = pd.read_csv(f'{caQTL_output_dir}{celltype}.epi', sep='\t', header=None)
    epi = epi.iloc[:,0:3]
    epi.columns = ['chr','prob','dis']
    #epi.loc[:,'chr'] = epi.loc[:,'prob'].str[3] WRONG
    epi['chr'] = epi['prob'].str.split(':', expand=True)[0]
    epi['chr'] = epi['chr'].str.replace('chr', '')
    epi.loc[:,'pos'] = vectorized_calculate_midpoint(epi['prob'])
    epi.loc[:,'gene_id'] = 'NA'
    epi.loc[:,'strand'] = 'NA'
    epi.to_csv(f'{caQTL_output_dir}{celltype}_update.epi', index=False, header=None, sep='\t')
    #linux command
    gi = f'{smr} --beqtl-summary {caQTL_output_dir}{celltype} --update-epi {caQTL_output_dir}{celltype}_update.epi'
    os.system(gi)

processing_Atypical_Bm_ITGAX
*******************************************************************
* Summary-data-based Mendelian Randomization (SMR)
* Version 1.3.1
* Build at Sep 21 2022 12:13:19, by GCC 8.3
* (C) 2015 Futao Zhang, Zhihong Zhu and Jian Yang
* The University of Queensland
* MIT License
*******************************************************************
Analysis started: 11:9:18,Sun Jan 19,2025

Options:
--eqtl-summary /CIMA/Data/SMR_caQTL_besd/result_for_besd//Atypical_Bm_ITGAX_caQTL_for_besd.txt
--matrix-eqtl-format 
--make-besd 
--out /CIMA/Data/SMR_caQTL_besd/besd/Atypical_Bm_ITGAX

Reading eQTL summary data from /CIMA/Data/SMR_caQTL_besd/result_for_besd//Atypical_Bm_ITGAX_caQTL_for_besd.txt ...
2174192 rows to be included from /CIMA/Data/SMR_caQTL_besd/result_for_besd//Atypical_Bm_ITGAX_caQTL_for_besd.txt.

Generating the .epi file...
603 probes have been saved in the file /CIMA/Data/SMR_caQTL_besd/besd/Atypical_Bm_ITGAX.epi.

Generating the .esi file...
1415785 SNP